# ChronoBound CM – Fluid Narrative with Starting Scene for New Timelines
This notebook now correctly injects a custom starting scene when a new tale is chosen.

In [1]:
# !pip install --upgrade openai
import openai
import getpass
import re
import random

openai.api_key = getpass.getpass('Enter your OpenAI API key: ')

Enter your OpenAI API key: ··········


In [2]:
# Define scenes and memory
new_scene = '''[SCENE START: A Temporal Breach on the Iron Coast]

The sky above the Iron Coast fractures like glass. From the rift spills a haze of violet mist, humming with the static of unraveling timelines.

You find yourself on the threshold of a breach — its center burning like a second sun. Temporal tidepools swirl at your feet, distorting the sand beneath your boots. Strange birds wheel overhead, their cries echoing forward and backward in time.

A voice, not quite yours, not quite alien, whispers: "Step forward, and forge the thread anew..."

'''
default_memory = [
    'Faction: Iron Thread',
    'Refused Echo Priestess’s offer',
    'Cursed by the Hollow God',
    'Entered Hollowstar'
]

In [5]:
# Session setup
scene_prompt = ''
memory_notes = []
scene_progression = []
in_combat = False
initiative_rolled = False
new_tale = False

choice = input('Would you like to continue your previous journey? (yes / no): ').strip().lower()
if choice in ['yes', 'y']:
    print('Resuming your previous story...\n')
    scene_prompt = '[SCENE START: The Vault Beneath Hollowstar]\nThe descent was silent... (continuing)'
    memory_notes = default_memory.copy()
    scene_progression.append('Resumed from Hollowstar Vault — Thread Guardian encounter')
else:
    print('Beginning a new tale through the timelines...\n')
    scene_prompt = new_scene
    memory_notes = default_memory.copy()
    scene_progression.append('Initiated new thread from Iron Coast Breach')
    new_tale = True
    print(scene_prompt)
    print('\nWhat do you do?')

Would you like to continue your previous journey? (yes / no): no
Beginning a new tale through the timelines...

[SCENE START: A Temporal Breach on the Iron Coast]

The sky above the Iron Coast fractures like glass. From the rift spills a haze of violet mist, humming with the static of unraveling timelines.

You find yourself on the threshold of a breach — its center burning like a second sun. Temporal tidepools swirl at your feet, distorting the sand beneath your boots. Strange birds wheel overhead, their cries echoing forward and backward in time.

A voice, not quite yours, not quite alien, whispers: "Step forward, and forge the thread anew..."



What do you do?


In [ ]:
# Main interactive loop
print("\nChronoBound CM — Fluid Interactive Session\nType 'exit' to quit.\n")

combat_keywords = ["attack", "draw weapon", "strike", "swing", "stab", "shoot", "fire", "fight", "roll initiative"]

def dynamic_scene_prompt():
    if new_tale and len(scene_progression) == 1:
        return "The rift pulses and time bends. What will you do?"
    last = scene_progression[-1]
    options = [
        f"The Vault responds subtly to your last move: {last}",
        f"Dust shifts and echoes your recent decision: {last}",
        f"The threads remember — {last} — and now?",
        f"Time hums a question after your act: {last}"
    ]
    return random.choice(options)

while True:
    print("\n" + dynamic_scene_prompt())
    player_input = input("What does the player do or say? ")
    if player_input.strip().lower() in ['exit', 'quit']:
        print("Exiting ChronoMaster session.")
        break

    scene_progression.append(player_input.strip())
    triggered_combat = any(re.search(rf"\b{k}\b", player_input.lower()) for k in combat_keywords)

    combat_injection = ''
    if triggered_combat and not in_combat:
        in_combat = True
        initiative_rolled = True
        player_init = random.randint(10, 20)
        enemy_init = random.randint(1, 15)
        combat_injection = f"""[COMBAT MODE ENGAGED]\nYou have awakened a threat within the timeline.\n🎲 Rolling for initiative…\nYou: {player_init}\nEnemy: {enemy_init}\nTime fractures. Battle begins."""
    elif in_combat and not triggered_combat:
        combat_injection = "[COMBAT CONTINUES] The enemy awaits your next move."

    full_prompt = f"""{scene_prompt}

[MEMORY NOTES]\n{chr(10).join(memory_notes)}

[PLAYER INPUT]\n{player_input}

{combat_injection}

[CM RESPONSE]"""

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "You are the ChronoMaster, an AI GM in a fractured multiverse. Speak with cinematic clarity, but keep responses concise and impactful. Use 2–4 short paragraphs max. Avoid over-explaining. Let the player drive the pace."
            },
            {
                "role": "user",
                "content": full_prompt
            }
        ],
        temperature=0.7,
        max_tokens=400
    )

    cm_reply = response.choices[0].message.content
    print("\nChronoMaster Responds:\n")
    print(cm_reply)
    print("\n" + "-"*80 + "\n")